In [ ]:
import netsquid as ns
# Set qstate formalism to use density matrices.
ns.set_qstate_formalism(ns.QFormalism.DM)

# Tutorial 2: Classical network simulation
## Network components

In [ ]:
from netsquid.nodes import Node

alice_node = Node("Alice")
bob_node   = Node("Bob")

In [ ]:
from netsquid.components import ClassicalChannel
from netsquid.components.models import FixedDelayModel
from netsquid.nodes import DirectConnection

# Classical messages & Qubits take 10ns to transmit
delay_model = FixedDelayModel(delay=10.)

channel_a_to_b = ClassicalChannel("Channel Alice to Bob",
                                  models={"delay_model": delay_model}
                                  )

channel_b_to_a = ClassicalChannel("Channel Bob to Alice",
                                  models={"delay_model": delay_model}
                                  )

connection = DirectConnection(name="connection",
                              channel_AtoB=channel_a_to_b,
                              channel_BtoA=channel_b_to_a)

In [ ]:
from netsquid.components import Port

alice_port = Port("Alice_port", component=alice_node)
bob_port   = Port("Bob_port",   component=bob_node)

alice_node.connect_to(remote_node=bob_node, connection=connection,
                     local_port_name="Alice_port", remote_port_name="Bob_port")

## Component diagram
![title](img/network_components_diagram.drawio.png)

# Protocols

### Create protocol classes

In [ ]:
from netsquid.protocols import NodeProtocol

class AliceProtocol(NodeProtocol):
    def __init__(self, node: Node, message: str):
        super().__init__(node)
        self.message = message

    def run(self):
        port = self.node.ports["Alice_port"]
        
        port.tx_output(self.message)
        print(f"{ns.sim_time()} ns Alice send message: {self.message}")


class BobProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Bob_port"]

        yield self.await_port_input(port)
        
        message = port.rx_input().items[0]
        
        print(f"{ns.sim_time()} ns Bob receives message {message}")

## Timeline diagram
![title](img/tutorial2_timeline.drawio.png)

### Create protocol instances

In [ ]:
message = "Hello from Alice"

alice_protocol = AliceProtocol(node=alice_node, message=message)
bob_protocol = BobProtocol(node=bob_node)

# Run simulation

In [ ]:
# Start all protocols
alice_protocol.start()
bob_protocol.start()

sim_stats = ns.sim_run()

print(sim_stats)

### Restart simulation

In [ ]:
alice_protocol.stop()
bob_protocol.stop()

# Clear scheduled events, entities and event handlers & Reset simulation time to 0
ns.sim_reset()

# Delay models

In [ ]:
from netsquid.components.models import FixedDelayModel, GaussianDelayModel, FibreDelayModel
from netsquid.components import ClassicalChannel

# T = 10 ns
fixed_delay_model = FixedDelayModel(delay=10.)

# T = Normal(mean=20ns, std=4s)
gaussian_delay_model = GaussianDelayModel(delay_mean=20, delay_std=4)

# T = c * length 
fibre_delay_model = FibreDelayModel(c=200000) # c = 200000 km/s

# length = 10 km
length = 10

channel_a_to_b = ClassicalChannel("Channel Alice to Bob",
                                  models={"delay_model": fibre_delay_model},
                                  length=length
                                  )


[Next tutorial](tutorial3.ipynb)